# 02 - Feature Engineering
## Real Estate Price Prediction

**Author:** Nicolas  
**Date:** 2025-01-09  
**Objective:** Créer et transformer les features pour optimiser la performance des modèles ML

---

### Table of Contents
1. [Data Loading](#1-data-loading)
2. [Outlier Treatment](#2-outlier-treatment)
3. [Feature Creation](#3-feature-creation)
4. [Encoding Categorical Variables](#4-encoding-categorical-variables)
5. [Feature Scaling](#5-feature-scaling)
6. [Target Transformation](#6-target-transformation)
7. [Feature Selection](#7-feature-selection)
8. [Train/Test Split](#8-train-test-split)
9. [Export Processed Data](#9-export-processed-data)

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, RobustScaler, LabelEncoder, OneHotEncoder
from scipy import stats
import warnings
import joblib

warnings.filterwarnings('ignore')

# Configuration
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
pd.set_option('display.max_columns', None)

%matplotlib inline

## 1. Data Loading

In [ ]:
# Charger les données
df = pd.read_csv('../data/real_estate_data.csv')

print(f"Dataset chargé: {df.shape[0]} lignes, {df.shape[1]} colonnes")
df.head()

## 2. Outlier Treatment

In [ ]:
# Fonction pour traiter les outliers
def cap_outliers_iqr(df, column, factor=1.5):
    """
    Cap les outliers en utilisant la méthode IQR.
    """
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - factor * IQR
    upper_bound = Q3 + factor * IQR
    
    # Cap values
    df[column] = df[column].clip(lower=lower_bound, upper=upper_bound)
    
    return df, lower_bound, upper_bound

# Traiter les outliers pour les variables clés
print("=" * 80)
print("TRAITEMENT DES OUTLIERS")
print("=" * 80)

outlier_cols = ['price', 'surface_m2', 'age_years']

for col in outlier_cols:
    before = df[col].describe()
    df, lower, upper = cap_outliers_iqr(df, col)
    after = df[col].describe()
    
    print(f"\n{col.upper()}:")
    print(f"  Bornes: [{lower:.2f}, {upper:.2f}]")
    print(f"  Avant: min={before['min']:.2f}, max={before['max']:.2f}")
    print(f"  Après: min={after['min']:.2f}, max={after['max']:.2f}")

## 3. Feature Creation

In [ ]:
print("=" * 80)
print("CRÉATION DE NOUVELLES FEATURES")
print("=" * 80)

# 1. Prix au m²
df['price_per_m2'] = df['price'] / df['surface_m2']

# 2. Surface par pièce
df['surface_per_room'] = df['surface_m2'] / df['rooms']

# 3. Age bins (catégories)
df['age_category'] = pd.cut(df['age_years'], 
                            bins=[-1, 5, 15, 30, 100],
                            labels=['Neuf', 'Récent', 'Moyen', 'Ancien'])

# 4. Surface bins
df['surface_category'] = pd.cut(df['surface_m2'],
                                bins=[0, 40, 70, 100, 1000],
                                labels=['Studio', 'Moyen', 'Grand', 'Très_grand'])

# 5. Score de confort (combinaison de features)
df['comfort_score'] = (df['has_elevator'].astype(int) + 
                       df['has_parking'].astype(int) + 
                       df['has_balcony'].astype(int))

# 6. Étage catégoriel
df['floor_category'] = df['floor'].apply(lambda x: 'RDC' if x == 0 else 
                                         ('Bas' if x <= 2 else 
                                         ('Moyen' if x <= 5 else 'Haut')))

# 7. Ratio rooms/surface (densité)
df['room_density'] = df['rooms'] / df['surface_m2']

# 8. Transformation log de la surface (pour linéariser la relation)
df['log_surface'] = np.log1p(df['surface_m2'])

# 9. Interaction ville × surface
df['city_surface_interaction'] = df['city'] + '_' + df['surface_category'].astype(str)

# 10. Feature polynomiale pour surface (capturer non-linéarité)
df['surface_squared'] = df['surface_m2'] ** 2

print(f"\n✅ {10} nouvelles features créées")
print(f"Dataset shape: {df.shape}")
print(f"\nNouvelles colonnes:")
new_cols = ['price_per_m2', 'surface_per_room', 'age_category', 'surface_category', 
            'comfort_score', 'floor_category', 'room_density', 'log_surface',
            'city_surface_interaction', 'surface_squared']
for col in new_cols:
    print(f"  - {col}")

In [ ]:
# Visualiser l'impact des nouvelles features
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Prix par catégorie d'âge
df.boxplot(column='price', by='age_category', ax=axes[0, 0])
axes[0, 0].set_title('Prix par Catégorie d\'Âge', fontsize=11, fontweight='bold')
axes[0, 0].set_xlabel('Catégorie d\'Âge')
axes[0, 0].set_ylabel('Prix (€)')

# Prix par catégorie de surface
df.boxplot(column='price', by='surface_category', ax=axes[0, 1])
axes[0, 1].set_title('Prix par Catégorie de Surface', fontsize=11, fontweight='bold')
axes[0, 1].set_xlabel('Catégorie de Surface')
axes[0, 1].set_ylabel('Prix (€)')

# Prix par score de confort
df.boxplot(column='price', by='comfort_score', ax=axes[1, 0])
axes[1, 0].set_title('Prix par Score de Confort', fontsize=11, fontweight='bold')
axes[1, 0].set_xlabel('Score de Confort (0-3)')
axes[1, 0].set_ylabel('Prix (€)')

# Log(surface) vs Prix
axes[1, 1].scatter(df['log_surface'], df['price'], alpha=0.5, s=10)
axes[1, 1].set_title('Prix vs Log(Surface)', fontsize=11, fontweight='bold')
axes[1, 1].set_xlabel('Log(Surface)')
axes[1, 1].set_ylabel('Prix (€)')

plt.tight_layout()
plt.show()

## 4. Encoding Categorical Variables

In [ ]:
print("=" * 80)
print("ENCODAGE DES VARIABLES CATÉGORIELLES")
print("=" * 80)

# Copie pour l'encodage
df_encoded = df.copy()

# 1. One-Hot Encoding pour 'city' (cardinal moyen)
city_dummies = pd.get_dummies(df_encoded['city'], prefix='city', drop_first=True)
df_encoded = pd.concat([df_encoded, city_dummies], axis=1)
print(f"\n✅ One-Hot Encoding: city → {len(city_dummies.columns)} nouvelles colonnes")

# 2. Ordinal Encoding pour 'energy_class' (ordre naturel: A > B > ... > G)
energy_mapping = {'A': 7, 'B': 6, 'C': 5, 'D': 4, 'E': 3, 'F': 2, 'G': 1}
df_encoded['energy_class_encoded'] = df_encoded['energy_class'].map(energy_mapping)
print(f"✅ Ordinal Encoding: energy_class → energy_class_encoded")

# 3. One-Hot Encoding pour les nouvelles catégories
for col in ['age_category', 'surface_category', 'floor_category']:
    dummies = pd.get_dummies(df_encoded[col], prefix=col, drop_first=True)
    df_encoded = pd.concat([df_encoded, dummies], axis=1)
    print(f"✅ One-Hot Encoding: {col} → {len(dummies.columns)} nouvelles colonnes")

# 4. Target Encoding pour 'city_surface_interaction' (haute cardinalité)
# Calculer la moyenne du prix par groupe
target_encoding = df_encoded.groupby('city_surface_interaction')['price'].mean().to_dict()
df_encoded['city_surface_encoded'] = df_encoded['city_surface_interaction'].map(target_encoding)
print(f"✅ Target Encoding: city_surface_interaction → city_surface_encoded")

print(f"\nShape après encodage: {df_encoded.shape}")

In [ ]:
# Sélectionner les features finales pour le modèle
# Exclure les colonnes originales et les colonnes non nécessaires
cols_to_drop = ['city', 'energy_class', 'age_category', 'surface_category', 
                'floor_category', 'city_surface_interaction', 'price_per_m2']

df_model = df_encoded.drop(columns=cols_to_drop, errors='ignore')

print("\n" + "=" * 80)
print("FEATURES FINALES POUR LE MODÈLE")
print("=" * 80)
print(f"\nNombre total de features: {df_model.shape[1] - 1}")
print(f"\nListe des features:")
feature_cols = [col for col in df_model.columns if col != 'price']
for idx, col in enumerate(feature_cols, 1):
    print(f"  {idx}. {col}")

## 5. Feature Scaling

In [ ]:
print("=" * 80)
print("NORMALISATION DES FEATURES")
print("=" * 80)

# Séparer X et y
X = df_model.drop('price', axis=1)
y = df_model['price']

# Identifier les colonnes numériques à scaler
# (exclure les colonnes one-hot encodées qui sont déjà 0/1)
numeric_cols = ['surface_m2', 'rooms', 'age_years', 'floor', 'comfort_score',
                'surface_per_room', 'room_density', 'log_surface', 'surface_squared',
                'city_surface_encoded', 'energy_class_encoded']

# Utiliser RobustScaler (résistant aux outliers)
scaler = RobustScaler()
X_scaled = X.copy()

# Scaler uniquement les colonnes numériques qui existent
cols_to_scale = [col for col in numeric_cols if col in X.columns]
X_scaled[cols_to_scale] = scaler.fit_transform(X[cols_to_scale])

print(f"\n✅ {len(cols_to_scale)} features normalisées avec RobustScaler")
print(f"\nFeatures scalées:")
for col in cols_to_scale:
    print(f"  - {col}")

# Sauvegarder le scaler pour la production
joblib.dump(scaler, '../models/scaler.pkl')
print("\n✅ Scaler sauvegardé: ../models/scaler.pkl")

## 6. Target Transformation

In [ ]:
# Tester la transformation log sur la cible
print("=" * 80)
print("TRANSFORMATION DE LA CIBLE")
print("=" * 80)

# Statistiques avant transformation
print("\nAvant transformation:")
print(f"  Skewness: {y.skew():.3f}")
print(f"  Kurtosis: {y.kurtosis():.3f}")

# Transformation log
y_log = np.log1p(y)

print("\nAprès transformation log:")
print(f"  Skewness: {y_log.skew():.3f}")
print(f"  Kurtosis: {y_log.kurtosis():.3f}")

# Visualisation
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# Distribution originale
axes[0].hist(y, bins=50, edgecolor='black', alpha=0.7)
axes[0].set_title(f'Distribution Originale (Skew: {y.skew():.2f})', 
                  fontsize=12, fontweight='bold')
axes[0].set_xlabel('Prix (€)')
axes[0].set_ylabel('Fréquence')

# Distribution log
axes[1].hist(y_log, bins=50, edgecolor='black', alpha=0.7, color='coral')
axes[1].set_title(f'Distribution Log-Transformée (Skew: {y_log.skew():.2f})', 
                  fontsize=12, fontweight='bold')
axes[1].set_xlabel('Log(Prix)')
axes[1].set_ylabel('Fréquence')

plt.tight_layout()
plt.show()

print("\n💡 La transformation log réduit le skewness et rend la distribution plus normale")
print("   → Utile pour les modèles linéaires (LinearRegression, Ridge, Lasso)")
print("   → Moins crucial pour les tree-based models (RandomForest, XGBoost)")

## 7. Feature Selection

In [ ]:
# Analyse de corrélation pour feature selection
print("=" * 80)
print("FEATURE SELECTION - ANALYSE DE CORRÉLATION")
print("=" * 80)

# Calculer corrélation avec la cible
correlations = X_scaled.corrwith(y).abs().sort_values(ascending=False)

print("\nTop 15 features les plus corrélées avec le prix:")
print(correlations.head(15))

# Visualisation
fig, ax = plt.subplots(figsize=(10, 8))
correlations.head(20).plot(kind='barh', ax=ax, edgecolor='black', alpha=0.7)
ax.set_title('Top 20 Features - Corrélation avec le Prix', fontsize=12, fontweight='bold')
ax.set_xlabel('Corrélation Absolue')
ax.set_ylabel('Features')
plt.tight_layout()
plt.show()

# Identifier les features à faible corrélation (potentiellement à supprimer)
low_corr_features = correlations[correlations < 0.05]
print(f"\n⚠️  {len(low_corr_features)} features avec corrélation < 0.05:")
if len(low_corr_features) > 0:
    print(low_corr_features)
    print("\n💡 Considérer de retirer ces features pour simplifier le modèle")

In [ ]:
# Matrice de corrélation entre features (détecter multicollinéarité)
corr_matrix = X_scaled.corr().abs()

# Trouver les paires de features très corrélées (> 0.9)
high_corr_pairs = []
for i in range(len(corr_matrix.columns)):
    for j in range(i+1, len(corr_matrix.columns)):
        if corr_matrix.iloc[i, j] > 0.9:
            high_corr_pairs.append((
                corr_matrix.columns[i], 
                corr_matrix.columns[j], 
                corr_matrix.iloc[i, j]
            ))

if high_corr_pairs:
    print("\n" + "=" * 80)
    print("⚠️  FEATURES HAUTEMENT CORRÉLÉES (Multicollinéarité)")
    print("=" * 80)
    for feat1, feat2, corr_val in high_corr_pairs:
        print(f"  {feat1} <-> {feat2}: {corr_val:.3f}")
    print("\n💡 Considérer de supprimer l'une des deux features dans chaque paire")
else:
    print("\n✅ Pas de multicollinéarité détectée (corrélation > 0.9)")

## 8. Train/Test Split

In [ ]:
print("=" * 80)
print("SPLIT TRAIN/TEST")
print("=" * 80)

# Split 80/20 avec stratification par ville (si possible)
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, 
    test_size=0.2, 
    random_state=42
)

# Aussi créer les versions log-transformées de y
y_train_log = np.log1p(y_train)
y_test_log = np.log1p(y_test)

print(f"\n✅ Split effectué:")
print(f"  Train set: {X_train.shape[0]} samples ({X_train.shape[0]/len(X_scaled)*100:.1f}%)")
print(f"  Test set:  {X_test.shape[0]} samples ({X_test.shape[0]/len(X_scaled)*100:.1f}%)")
print(f"  Number of features: {X_train.shape[1]}")

# Statistiques
print(f"\nStatistiques Train set:")
print(f"  Prix moyen: {y_train.mean():,.2f} €")
print(f"  Prix médian: {y_train.median():,.2f} €")
print(f"  Écart-type: {y_train.std():,.2f} €")

print(f"\nStatistiques Test set:")
print(f"  Prix moyen: {y_test.mean():,.2f} €")
print(f"  Prix médian: {y_test.median():,.2f} €")
print(f"  Écart-type: {y_test.std():,.2f} €")

## 9. Export Processed Data

In [ ]:
print("=" * 80)
print("SAUVEGARDE DES DONNÉES PRÉPARÉES")
print("=" * 80)

# Sauvegarder les datasets
import pickle

# Créer un dictionnaire avec tous les datasets
data_dict = {
    'X_train': X_train,
    'X_test': X_test,
    'y_train': y_train,
    'y_test': y_test,
    'y_train_log': y_train_log,
    'y_test_log': y_test_log,
    'feature_names': X_train.columns.tolist(),
    'scaler': scaler
}

# Sauvegarder
with open('../data/processed_data.pkl', 'wb') as f:
    pickle.dump(data_dict, f)

print("\n✅ Données sauvegardées: ../data/processed_data.pkl")
print("\nContenu du fichier:")
print("  - X_train, X_test (features normalisées)")
print("  - y_train, y_test (cible originale)")
print("  - y_train_log, y_test_log (cible transformée)")
print("  - feature_names (liste des features)")
print("  - scaler (pour la production)")

# Sauvegarder aussi en CSV pour inspection
train_df = pd.concat([X_train, y_train], axis=1)
test_df = pd.concat([X_test, y_test], axis=1)

train_df.to_csv('../data/train_processed.csv', index=False)
test_df.to_csv('../data/test_processed.csv', index=False)

print("\n✅ Également sauvegardé en CSV:")
print("  - ../data/train_processed.csv")
print("  - ../data/test_processed.csv")

---
## Summary

### Features créées:
1. **Numerical transformations:**
   - `price_per_m2`: Prix au m²
   - `surface_per_room`: Surface par pièce
   - `log_surface`: Transformation log de la surface
   - `surface_squared`: Surface au carré (polynomial)
   - `room_density`: Densité (rooms/surface)

2. **Categorical features:**
   - `age_category`: Catégories d'âge (Neuf/Récent/Moyen/Ancien)
   - `surface_category`: Catégories de surface (Studio/Moyen/Grand/Très_grand)
   - `floor_category`: Catégories d'étage (RDC/Bas/Moyen/Haut)
   - `comfort_score`: Score 0-3 (elevator + parking + balcony)

3. **Interactions:**
   - `city_surface_encoded`: Interaction ville × surface (target encoded)

### Encodings appliqués:
- One-Hot Encoding: city, age_category, surface_category, floor_category
- Ordinal Encoding: energy_class (A=7 ... G=1)
- Target Encoding: city_surface_interaction

### Preprocessing:
- Outliers: cappés avec méthode IQR
- Scaling: RobustScaler sur features numériques
- Target: log transformation disponible (optionnel)

### Next Step:
**Notebook 03 - Model Training**: Entraîner et comparer différents modèles ML

---